<a href="https://colab.research.google.com/github/s0ye0nyang/News-Recomendation-System/blob/main/news_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Selenium

!apt-get update # to update ubuntu to correctly run apt install

!apt install chromium-chromedriver

In [ ]:
import torch
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정

chrome_options.add_argument('--no-sandbox')

chrome_options.add_argument('--disable-dev-shm-usage')

# setup Driver|Chrome : 크롬드라이버를 사용하는 driver 생성
driver= webdriver.Chrome('chromedriver',options=chrome_options)
driver.implicitly_wait(3) # 암묵적으로 웹 자원을 (최대) 3초 기다리기
date = 10
news_df = pd.DataFrame()
max_page = 10
for i in range(3): # categories
  for page in range(max_page):
    for j in range(2,10): # news per page
      URL = f"https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=10{i}&date=20201210&page={page}"
      driver.get(URL) 
      html = driver.page_source 
      soup = BeautifulSoup(html, 'html.parser')
      single_news_link = [a['href'] for a in soup.select(f'#main_content > div.list_body.newsflash_body > ul.type06_headline > li:nth-of-type({j}) > dl > dt:nth-of-type(2) > a[href]')]
      if single_news_link:
        # print("i,j:",i,j)
        driver.get(single_news_link[0])
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        news_contents = soup.select('#articleBodyContents')[0].get_text().replace('\n','')
        news_df = news_df.append({'제목':single_news_link, 
                                  '기사':news_contents},ignore_index=True)
    

display(news_df)
  


In [ ]:
news_df.to_csv("naver_news.csv", index=False)